In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Activation
from keras.utils import get_custom_objects
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot

np.random.seed(7)

Using TensorFlow backend.


In [2]:
#version check
print('Python version : ', sys.version)
print('TensorFlow version : ', tf.__version__)
print('Keras version : ', keras.__version__)

Python version :  3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
TensorFlow version :  1.13.1
Keras version :  2.2.4


In [14]:
def modified_sigmoid(x):
    return 0.5+(x**5/2700000)-((87*(x**3))/27000)+(54025*x/2700000)
def modified_relu(x):
    return 0.66+x/2+x**2/8.5+x**4/600
def modified_tanh(x):
    return ((23*x)/25)+x**3/10-((7*(x**5))/25)

get_custom_objects().update({"modified_sigmoid" : Activation(modified_sigmoid)})
get_custom_objects().update({"modified_relu" : Activation(modified_relu)})
get_custom_objects().update({"modified_tanh" : Activation(modified_tanh)})

In [15]:
#MNIST
img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 12

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [22]:
#CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1000, activation='modified_tanh')) #activation function
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 64)        8256      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1000)              3137000   
__________

In [23]:
#training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1, 
                 validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 120s 2ms/step - loss: 0.1676 - acc: 0.9483 - val_loss: 0.0518 - val_acc: 0.9833
Epoch 2/12
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0605 - acc: 0.9808 - val_loss: 0.0455 - val_acc: 0.9852
Epoch 3/12
60000/60000 [==============================] - 114s 2ms/step - loss: 0.0449 - acc: 0.9861 - val_loss: 0.0273 - val_acc: 0.9904
Epoch 4/12
60000/60000 [==============================] - 106s 2ms/step - loss: 0.0388 - acc: 0.9881 - val_loss: 0.0282 - val_acc: 0.9899
Epoch 5/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0336 - acc: 0.9893 - val_loss: 0.0282 - val_acc: 0.9907
Epoch 6/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0296 - acc: 0.9903 - val_loss: 0.0286 - val_acc: 0.9906
Epoch 7/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0281 - acc: 0.9913 - val_loss: 0.0258 - 

In [24]:
#evaluate
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.026457260819714064
Test accuracy: 0.9918
